# Kernel chido 

In [1]:
using CUDA, LinearAlgebra, Statistics

In [2]:
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]
#A\b = [-1 0 2]

M = Diagonal(A) # Tal vez esta es una mejor forma(?)

N = M - A

T = inv(M) * N

f = inv(M) * b

nT, mT = size(T)

S = zeros(Int64, nT)

[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]

P = zeros(nT, mT)
[P[i,j] = 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0];
#0.0  1.0  0.0
#0.5  0.0  0.5
#0.5  0.5  0.0

[P[i,j] = sum(P[i,1:j]) for i = 1:nT, j = 1:mT if P[i,j] != 0];
#0.0 1.0 0.0
#0.5 0.0 1.0
#0.5 1.0 0.0

In [3]:
T

3×3 Array{Float64,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

In [4]:
P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.5  0.0  1.0
 0.5  1.0  0.0

In [5]:
e = 10
 # Aquí cambiamos la posición del vector solución que queremos aproximar, ya sea X[1], X[2] o X[3]
var = 1 

u = rand(e)
Point = [var]
Nextpoint = []
#nextpoint = [u[i] for i = 1:40, while u]
for tamaño = 1:e
    np = 1
    while u[tamaño] >= P[Point[tamaño], np]
        np += 1
    end
    push!(Nextpoint, np)
    push!(Point, np)
end

In [6]:
d_W = CUDA.ones(Float64, e) # Aquí suponemos W_0 = 1.
d_point = CuArray(Point)
d_nextpoint = CuArray{Int64}(Nextpoint)
d_W_new = CuArray{Float64}(undef, e)
d_P = CuArray{Float64}(P)
d_T = CuArray{Float64}(T)
d_X = CUDA.fill(Float64(f[var]), e)
d_f = CuArray(Float64.(f))

function kernel(W, W_new, X, T, P, nextpoint, point, f)
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    #W_new = W *(T[point, nextpoint]/P[point, nextpoint])
    #X[column_T] += W_new * f[nextpoint] # ------------------------Aquí
    #W = W_new
    #point = nextpoint
    W_new[i] = W[i] * CUDA.pow((T[point[i], nextpoint[i]]/P[point[i], nextpoint[i]]), i)
    @cushow W_new[i]
    if W_new[i] >= 0.05
        X[i] +=  W_new[i] * f[nextpoint[i]]
    end
    return
end

@cuda threads = e kernel(d_W, d_W_new, d_X, d_T, d_P, d_nextpoint, d_point, d_f)

mean(Array(d_X))

┌ Error: Could not initialize CUDA
│   exception = (CuError(CUDA.CUDA_ERROR_UNKNOWN, nothing), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007fe72c88d48e, Ptr{Nothing} @0x00007fe72c8c3f18, Ptr{Nothing} @0x00007fe72c8e303c, Ptr{Nothing} @0x00007fe72c8c24ef, Ptr{Nothing} @0x00007fe72c8c2674, Ptr{Nothing} @0x00007fe72c88ceb1, Ptr{Nothing} @0x00007fe72c88cfb8, Ptr{Nothing} @0x00007fe72c8bb2f9, Ptr{Nothing} @0x00007fe72c8bb5fa, Ptr{Nothing} @0x00007fe72c8bc772, Ptr{Nothing} @0x00007fe72c8c1ddc, Ptr{Nothing} @0x00007fe72c8c1e8c, Ptr{Nothing} @0x00007fe758923082, Ptr{Nothing} @0x00007fe72c88cdad, Ptr{Nothing} @0x00007fe72c8bf6a5, Ptr{Nothing} @0x00007fe72c8bf99a, Ptr{Nothing} @0x00007fe72c8c03fa, Ptr{Nothing} @0x00007fe72c8be290, Ptr{Nothing} @0x00007fe72c8be635, Ptr{Nothing} @0x00007fe758923082, Ptr{Nothing} @0x00007fe72c8bdbdc, Ptr{Nothing} @0x00007fe72c8ba6fd, Ptr{Nothing} @0x00007fe72c8ba713, Ptr{Nothing} @0x00007fe758923082, Ptr{Nothing} @0x00007fe72c8ba675, Ptr{Nothing} @0

AssertionError: AssertionError: CUDA.jl did not successfully initialize, and is not usable.

In [75]:
X[1]

0.5127022f0